In [1]:
from torch.utils.data import Dataset, DataLoader

import hyperparameters
from preprocessing import TokenConv, wer
from utils import LipDataset
from model import LipNet

trainset = LipDataset('./dataset')
trainloader = DataLoader(
        trainset,
        batch_size=2,
        shuffle=False,
    )

Dataset loaded successfully!


In [2]:
import torch
crit = torch.nn.CTCLoss(blank=0, zero_infinity=True)
model = LipNet()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epoch_loss = 0
ctcdecoder = TokenConv()
for i, (vid, align, vid_len, align_len) in enumerate(trainloader):
    y = model(vid)
    # print(vid_len, align_len)
    # print(y.permute(1, 0, 2).shape) #torch.Size([2, 75, 28])
    loss = crit(
        y.permute(1, 0, 2),
        align,
        vid_len.view(-1),
        align_len.view(-1),
    )
    y = torch.argmax(y, dim=2)
    if hyperparameters.debug:
        # print("Model out : ", y)
        print("Loss : ", loss.item())
        if torch.any(torch.isnan(y)).item():
            print("Pred labels have nan")
            exit()
        if torch.isnan(loss):
            print("Only loss is nan")
            exit()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
    for tru, pre in zip(align.tolist(), y.tolist()):
        true_txt = ctcdecoder.ctc_decode(tru)
        pred_txt = ctcdecoder.ctc_decode(pre)
        print("True: ", true_txt)
        print("Pred: ", pred_txt)

                # train_wer.extend(wer(pred_txt, true_txt))
                # if epoch % hyperparameters.display:
                    # print(f"Epoch [GPU:{gpu_id}]: ", epoch)

torch.Size([2, 3, 75, 50, 100])
Loss :  7.101967811584473
True:  bin white with a nine again
Pred:  lvlpspvlvspcivqvcvqvlvqwvqvnvivqkvlwafwlavwv
True:  bin gren at a six please
Pred:  ekvlkakqlakakaqkadamqdlqlakaqakavndadnadvkalajavkavazadkdad
torch.Size([2, 3, 75, 50, 100])
Loss :  6.32188606262207
True:  set red sp with i zero now
Pred:  acwdaacatada sandvadanadadvhaldampvlktaraasnnasabh
True:  place blue by i eight sp please
Pred:  genaqkgkioggaozsosegqogqaongqtcjvaqagksiqglgnqgaqvsasgqklgqainik
torch.Size([2, 3, 75, 50, 100])
Loss :  6.650384902954102
True:  set blue by n seven again
Pred:  clkifafckyidaokakjkjfemkkizpehakhfdkadikcxakmtoilaiaahkjhkamja
True:  bin red sp with g seven again
Pred:  oklilxyvavnavxnfpaxlflbuanfweofnxivotgbsofeeo fofvnkeoneolblelsvo
torch.Size([2, 3, 75, 50, 100])
Loss :  6.794860363006592
True:  bin white with a six now
Pred:  zmvtxtlakglavxqcdxgxsvaagvaxaqaxalaqqdxasvxaxcnanasapg
True:  set red sp by u five son
Pred:  atakakdyaadaaaaapaaatakaaaaha
torc

KeyboardInterrupt: 